In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/usa-housing/USA_Housing.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
sns.pairplot(df)

In [ ]:
df.corr()

In [ ]:
adrs=df['Address']
adrs

In [ ]:
df.drop( 'Address' , inplace=True , axis=1)
df.info()

In [ ]:
x = df.drop('Price', axis=1)
y= df['Price']

In [ ]:
x.shape

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
polynomial_converter=PolynomialFeatures(degree=3, include_bias=False)

In [ ]:
poly_features=polynomial_converter.fit(x)

In [ ]:
poly_features=polynomial_converter.transform(x)

In [ ]:
poly_features.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.25, random_state=101)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
pmodel=LinearRegression()

In [ ]:
pmodel.fit(x_train, y_train)

In [ ]:
y_pred=pmodel.predict(x_test)

In [ ]:
from sklearn import metrics

In [ ]:
MAE= metrics.mean_absolute_error(y_test, y_pred)
MSE= metrics.mean_squared_error(y_test, y_pred)
RMSE= np.sqrt(MSE)

In [ ]:
pd.DataFrame([MAE, MSE, RMSE], index=['MAE', 'MSE', 'RMSE'], columns=['MATRIX'])

In [ ]:
Residuals = pd.DataFrame(y_test - y_pred)
Residuals

In [ ]:
residuals = y_test - y_pred

In [ ]:
residuals.mean()

In [ ]:
100*(residuals.mean()/df['Price'].mean())

In [ ]:
Residuals.describe()

In [ ]:
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel('Y-Test')
plt.ylabel('Y-Pred')

In [ ]:
sns.scatterplot(x=y_test, y=residuals)

In [ ]:
sns.displot(residuals , bins=50)

In [ ]:
XS_train, XS_test, ys_train, ys_test = train_test_split(x, y, test_size=0.25, random_state=101)

In [ ]:
simplemodel=LinearRegression()
simplemodel.fit(XS_train, ys_train)
ys_pred=simplemodel.predict(XS_test)

In [ ]:
MAE_simple = metrics.mean_absolute_error(ys_test,ys_pred)
MSE_simple = metrics.mean_squared_error(ys_test,ys_pred)
RMSE_simple = np.sqrt(MSE_simple)

In [ ]:
pd.DataFrame({'Poly Metrics': [MAE, MSE, RMSE], 'Simple Metrics':[MAE_simple, MSE_simple, RMSE_simple]}, index=['MAE', 'MSE', 'RMSE'])

In [ ]:
train_RMSE_list=[]
test_RMSE_list=[]

In [ ]:
for d in range(1,10):
    polynomial_converter= PolynomialFeatures(degree=d, include_bias=False)
    poly_features= polynomial_converter.fit(x)
    poly_features= polynomial_converter.transform(x)
    
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.25, random_state=101)
    
    polymodel=LinearRegression()
    polymodel.fit(X_train, y_train)
    
    y_train_pred=polymodel.predict(X_train)
    y_test_pred=polymodel.predict(X_test)
    
    train_RMSE=np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
    test_RMSE=np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
    
    train_RMSE_list.append(train_RMSE)
    test_RMSE_list.append(test_RMSE)

In [ ]:
train_RMSE_list

In [ ]:
test_RMSE_list

In [ ]:
plt.plot(range(1,10), train_RMSE_list[:9], label='Train RMSE')
plt.plot(range(1,10), test_RMSE_list[:9], label='Test RMSE')

plt.xlabel('Polynomial Degree')
plt.ylabel('RMSE')
plt.legend()

In [ ]:
plt.plot(range(1,6), train_RMSE_list[:5], label='Train RMSE')
plt.plot(range(1,6), test_RMSE_list[:5], label='Test RMSE')

plt.xlabel('Polynomial Degree')
plt.ylabel('RMSE')
plt.legend()

In [ ]:
Final_polynomial_converter= PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
Final_poly_features= Final_polynomial_converter.fit(x)
Final_poly_features= Final_polynomial_converter.transform(x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Final_poly_features, y, test_size=0.33, random_state=101)

In [ ]:
Final_PolyModel=LinearRegression()
Final_PolyModel.fit(X_train, y_train)

In [ ]:
from joblib import dump, load

In [ ]:
dump(Final_PolyModel, 'Price_PolyModel.joblib')

In [ ]:
dump(Final_polynomial_converter, 'Poly_Converter.joblib')

In [ ]:
Load_Converter=load('Poly_Converter.joblib')
Load_PolyModel=load('Price_PolyModel.joblib')

In [ ]:
#Avg. Area Income :100000  Avg. Area House Age:3  Avg. Area Number of Rooms :4    Avg. Area Number of Bedrooms: 2   Area Population: 20000
campaign=[[100000 , 3 , 4 , 2 , 20000]]

In [ ]:
campaign_poly=Load_Converter.transform(campaign)

In [ ]:
Final_PolyModel.predict(campaign_poly)